In [1]:
from google.colab import drive
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
from sklearn.model_selection import train_test_split

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Step 1: Load the trained model
model_path = '/content/drive/MyDrive/30000/best_30000_17db_model.h5'
model = load_model(model_path)

In [3]:
# Step 2: Load training and validation data
X_train = np.load('/content/drive/MyDrive/30000/merged_signals_30000.npy')
y_train = np.load('/content/drive/MyDrive/30000/merged_labels_30000.npy')

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

print(f"Training Data Shape: {X_train.shape}")
print(f"Validation Data Shape: {X_val.shape}")


Training Data Shape: (48000, 36, 1000, 2)
Validation Data Shape: (12000, 36, 1000, 2)


In [8]:
# Step 3: Quantization functions
def convert_to_tflite_bf16(model, X_train):
    def representative_dataset_generator():
        for i in range(len(X_train)):
            yield [np.expand_dims(X_train[i].astype(np.float32), axis=0)]

    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.representative_dataset = representative_dataset_generator
    converter.target_spec.supported_types = [tf.float16]
    return converter.convert()

def convert_to_tflite_fp16(model):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.target_spec.supported_types = [tf.float16]
    return converter.convert()

def convert_to_tflite_int8(model, X_train):
    def representative_dataset_generator():
        for i in range(len(X_train)):
            yield [np.expand_dims(X_train[i].astype(np.float32), axis=0)]

    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.representative_dataset = representative_dataset_generator
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    converter.inference_input_type = tf.int8
    converter.inference_output_type = tf.int8
    return converter.convert()


In [9]:
# Step 4: Save the converted TFLite models
print("Converting and saving quantized models...")

bf16_tflite_model = convert_to_tflite_bf16(model, X_train)
bf16_path = '/content/drive/MyDrive/30000/best_30000_17db_model_bf16.tflite'
with open(bf16_path, 'wb') as f:
    f.write(bf16_tflite_model)

fp16_tflite_model = convert_to_tflite_fp16(model)
fp16_path = '/content/drive/MyDrive/30000/best_30000_17db_model_fp16.tflite'
with open(fp16_path, 'wb') as f:
    f.write(fp16_tflite_model)

int8_tflite_model = convert_to_tflite_int8(model, X_train)
int8_path = '/content/drive/MyDrive/30000/best_30000_17db_model_int8.tflite'
with open(int8_path, 'wb') as f:
    f.write(int8_tflite_model)

print(f"BF16 model saved to {bf16_path}")
print(f"FP16 model saved to {fp16_path}")
print(f"INT8 model saved to {int8_path}")

Converting and saving quantized models...


/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


BF16 model saved to /content/drive/MyDrive/30000/best_30000_17db_model_bf16.tflite
FP16 model saved to /content/drive/MyDrive/30000/best_30000_17db_model_fp16.tflite
INT8 model saved to /content/drive/MyDrive/30000/best_30000_17db_model_int8.tflite


In [10]:
# Step 5: Evaluate the TFLite models
def evaluate_tflite_model(tflite_model_path, X_val, y_val):
    interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    correct_predictions = 0

    for i in range(len(X_val)):
        input_data = np.expand_dims(X_val[i], axis=0).astype(input_details[0]['dtype'])
        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        output_data = interpreter.get_tensor(output_details[0]['index'])

        predicted_label = (output_data >= 0.5).astype(int)
        true_label = y_val[i]

        if len(true_label.shape) > 1:
            true_label = np.argmax(true_label)

        if predicted_label == true_label:
            correct_predictions += 1

    accuracy = correct_predictions / len(X_val)
    return accuracy

In [11]:
# Compare accuracies
print("Evaluating quantized models...")

bf16_accuracy = evaluate_tflite_model(bf16_path, X_val, y_val)
fp16_accuracy = evaluate_tflite_model(fp16_path, X_val, y_val)
int8_accuracy = evaluate_tflite_model(int8_path, X_val, y_val)

print("\nQuantization Accuracy Comparison:")
print(f"BF16 Accuracy: {bf16_accuracy:.4f}")
print(f"FP16 Accuracy: {fp16_accuracy:.4f}")
print(f"INT8 Accuracy: {int8_accuracy:.4f}")


Evaluating quantized models...

Quantization Accuracy Comparison:
BF16 Accuracy: 0.7522
FP16 Accuracy: 0.7522
INT8 Accuracy: 0.4998
